<a href="https://colab.research.google.com/github/bhstoller/multi-agent-customer-service/blob/main/agent_to_agent_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Multi-Agent Customer Service System - A2A Demo**

This notebook demonstrates agent-to-agent (A2A) communication with three specialized agents coordinating to handle customer service queries.

## System Components
- **Customer Data Agent**: Interfaces with MCP server for customer/ticket operations
- **Support Agent**: Handles customer support questions and escalations
- **Router Orchestrator**: Coordinates between agents based on query intent

## Prerequisites
1. MCP server running (in separate Colab or local)
2. Google API key in Colab Secrets (as 'a5-key')
3. MCP_SERVER_URL in Colab Secrets (ngrok or localhost)

## Clone Repository and Setup

Clone the Github repository to get access to all modular code (config.py, agents.py, router.py, mcp_server.py), configuration files, and python requirements.

In [1]:
!git clone https://github.com/bhstoller/multi-agent-customer-service.git

Cloning into 'multi-agent-customer-service'...
remote: Enumerating objects: 275, done.
remote: Counting objects: 100% (110/110), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 275 (delta 66), reused 71 (delta 41), pack-reused 165 (from 1)
Receiving objects: 100% (275/275), 149.86 KiB | 9.99 MiB/s, done.
Resolving deltas: 100% (139/139), done.


In [2]:
!pip install -q -r multi-agent-customer-service/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.3/130.3 kB 5.5 MB/s eta 0:00:00


In [3]:
# Add repo to path
import sys
sys.path.insert(0, '/content/multi-agent-customer-service/src')

## Configure Notebook

In [8]:
print("Configuring notebook...")
# Import configuration libraries
from config import LOG_LEVEL, LLM_MODEL, CUSTOMER_DATA_URL, SUPPORT_URL
from google.colab import userdata
from pathlib import Path
import logging
import os
import sys

# Set Google Cloud configuration
os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'FALSE'
os.environ['GOOGLE_CLOUD_PROJECT'] = 'app-ai-a5'
os.environ['GOOGLE_CLOUD_LOCATION'] = 'us-central1'
os.environ['GOOGLE_API_KEY'] = userdata.get('a5-key')
print("- Google API Key loaded")

# Set the LLM model
print(f"- LLM Model loaded: {LLM_MODEL}")

# Set MCP Server configuration
os.environ['MCP_SERVER_URL'] = userdata.get('MCP_SERVER_URL')
print(f"- MCP Server URL loaded: {os.environ['MCP_SERVER_URL']}")

# Reload config so it picks up the new MCP Server
import importlib
import config
importlib.reload(config)

# Set up logging
log_level = getattr(logging, LOG_LEVEL)
logging.basicConfig(level=log_level, force=True)
logging.getLogger('asyncio').setLevel(log_level)
logging.getLogger('aiohttp').setLevel(log_level)
print(f"- Logging level loaded: {LOG_LEVEL}")

print("Configuration complete")

Configuring notebook...
- Google API Key loaded
- LLM Model loaded: gemini-2.0-flash
- MCP Server URL loaded: https://polar-nonsolubly-madden.ngrok-free.dev/mcp
- Logging level loaded: CRITICAL
Configuration complete


## Import Dependencies

In [9]:
# Handle A2A SDK import issue
from a2a.client import client as real_client_module
from a2a.client.card_resolver import A2ACardResolver

class PatchedClientModule:
    def __init__(self, real_module) -> None:
        for attr in dir(real_module):
            if not attr.startswith('_'):
                setattr(self, attr, getattr(real_module, attr))
        self.A2ACardResolver = A2ACardResolver

patched_module = PatchedClientModule(real_client_module)
sys.modules['a2a.client.client'] = patched_module

In [11]:
print("Importing libraries...")
# Import all modular code
# from config import MCP_SERVER_URL, CUSTOMER_DATA_URL, SUPPORT_URL, LLM_MODEL, LOG_LEVEL
from agents import customer_data_agent, customer_data_agent_card, support_agent, support_agent_card
from router import RouterOrchestrator, A2ASimpleClient
from mcp_server import create_app

# Import A2A infrastructure libraries
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.artifacts import InMemoryArtifactService
from google.adk.memory.in_memory_memory_service import InMemoryMemoryService
from google.adk.sessions import InMemorySessionService
from a2a.server.apps import A2AStarletteApplication
from a2a.server.request_handlers import DefaultRequestHandler
from a2a.server.tasks import InMemoryTaskStore
from google.adk.a2a.executor.a2a_agent_executor import A2aAgentExecutor, A2aAgentExecutorConfig
from a2a.utils.constants import AGENT_CARD_WELL_KNOWN_PATH
import uvicorn
import nest_asyncio

# Import general python libraries
import asyncio
import threading
import time

# Turn off warnings for readability
import warnings
warnings.filterwarnings('ignore')

print("All imports loaded")

Importing libraries...
All imports loaded


### Start MCP Server

In [12]:
print("Starting the MCP Server...")
MCP_SERVER_HOST = "127.0.0.1"
MCP_SERVER_PORT = "10050"

def run_mcp_server_in_background():
    app = create_app()
    app.run(
        host= MCP_SERVER_HOST,
        port= MCP_SERVER_PORT,
        debug= False,
        use_reloader= False
    )

mcp_thread = threading.Thread(
    target= run_mcp_server_in_background,
    daemon= True
)
mcp_thread.start()
time.sleep(2)
print(f'MCP server started on http://{MCP_SERVER_HOST}:{MCP_SERVER_PORT}')

Starting the MCP Server...
 * Serving Flask app 'mcp_server'
 * Debug mode: off
MCP server started on http://127.0.0.1:10050


## Start Agent Servers

Start Customer Data and Support agents on ports 10020 and 10021

In [13]:
def create_agent_a2a_server(agent, agent_card):
    """Create an A2A server for any ADK agent."""
    runner = Runner(
        app_name=agent.name,
        agent=agent,
        artifact_service=InMemoryArtifactService(),
        session_service=InMemorySessionService(),
        memory_service=InMemoryMemoryService(),
    )

    config = A2aAgentExecutorConfig()
    executor = A2aAgentExecutor(runner=runner, config=config)

    request_handler = DefaultRequestHandler(
        agent_executor=executor,
        task_store=InMemoryTaskStore(),
    )

    return A2AStarletteApplication(
        agent_card=agent_card, http_handler=request_handler
    )

In [14]:
print("Starting agent servers...")
async def run_agent_server(agent, agent_card, port) -> None:
    """Run a single agent server."""
    app = create_agent_a2a_server(agent, agent_card)

    config = uvicorn.Config(
        app.build(),
        host='127.0.0.1',
        port=port,
        log_level='warning',
        loop='none',
    )

    server = uvicorn.Server(config)
    await server.serve()


async def start_all_servers() -> None:
    """Start Customer Data and Support agent servers."""
    tasks = [
        asyncio.create_task(
            run_agent_server(customer_data_agent, customer_data_agent_card, 10020)
        ),
        asyncio.create_task(
            run_agent_server(support_agent, support_agent_card, 10021)
        ),
    ]

    await asyncio.sleep(2)
    print('Agent servers started:')
    print('   - Customer Data Agent: http://127.0.0.1:10020')
    print('   - Support Agent:       http://127.0.0.1:10021')

    try:
        await asyncio.Event().wait()
    except KeyboardInterrupt:
        print('Shutting down servers...')


def run_servers_in_background() -> None:
    """Run servers in a background thread."""
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    loop.run_until_complete(start_all_servers())


# Apply nest_asyncio before starting servers
nest_asyncio.apply()

# Start servers in background thread
server_thread = threading.Thread(target=run_servers_in_background, daemon=True)
server_thread.start()
time.sleep(3)
print("Agent servers initialized")

Starting agent servers...
Agent servers started:
   - Customer Data Agent: http://127.0.0.1:10020
   - Support Agent:       http://127.0.0.1:10021
Agent servers initialized


## Initialize The Router/Orchestrator Agent

In [15]:
# Create A2A client and router
a2a_client = A2ASimpleClient()
router = RouterOrchestrator(a2a_client)

print("Router Orchestrator initialized and ready for queries")

Router Orchestrator initialized and ready for queries


## Test Scenarios

Run three test scenarios demonstrating A2A coordination:

### Scenario 1: Simple Query (Single Agent)

Query: Get customer information for a specific ID

Expected Flow:
1. Router analyzes query
2. Router calls Customer Data Agent
3. Returns customer details

In [16]:
query_1 = "Get customer information for customer ID 5"

result_1 = await router.process_query(query_1)
print(f"\nFINAL RESPONSE:\n{result_1}")


 USER QUERY: Get customer information for customer ID 5 

[ROUTER STEP 1]: The user is asking for customer information for a specific customer ID. I should use the customer_data agent to retrieve this information.
    >>> [A2A CALL] Connecting to customer_data at http://localhost:10020...
   <<< [A2A RESPONSE]: [CUSTOMER_DATA_AGENT]: Calling get_customer for ID 5 ```json {   "customer": {     "id": 5,     "name": "Charlie Brown",     "email": "charlie.brown@email.com",     "phone": "+1-555-0105",     "status...

[ROUTER STEP 2]: I have retrieved the customer information from the customer_data agent. Now I can respond to the user.

[ROUTER]: Task Complete.

FINAL RESPONSE:
Customer information for customer ID 5:
Name: Charlie Brown
Email: charlie.brown@email.com
Phone: +1-555-0105
Status: active
Created at: 2025-11-17 21:35:02
Updated at: 2025-11-17 21:35:02


### Scenario 2: Negotiation/Escalation (Multiple Agents)

Query: Customer wants to cancel subscription but has billing issues

Expected Flow:
1. Router detects multiple intents
2. Router calls Support Agent (for cancellation policy)
3. Router calls Customer Data Agent (for billing history)
4. Router synthesizes coordinated response

In [ ]:
query_2 = "I want to cancel my subscription but I'm having billing issues. My customer ID is 1."

result_2 = await router.process_query(query_2)
print(f"\nFINAL RESPONSE:\n{result_2}")


 USER QUERY: I want to cancel my subscription but I'm having billing issues. My customer ID is 1. 

[ROUTER STEP 1]: The user wants to cancel their subscription and is having billing issues. To best assist, I need to understand the billing issues and whether cancellation is the appropriate next step. I will first call the customer_data agent to gather information about customer 1 and their ticket history to understand the billing issues.
   >>> [A2A CALL] Connecting to customer_data at http://localhost:10020...
   <<< [A2A RESPONSE]: [CUSTOMER_DATA_AGENT]: Calling get_customer for ID 1 ```json {"customer": { "id": 1, "name": "John Doe", "email": "john.doe@example.com", "phone": "+1-555-0101", "status": "active", "created_at": "202...

[ROUTER STEP 2]: The customer is active, has multiple open tickets about cancellation requests and billing issues. I will now call the support_agent to decide on the next course of action.
   >>> [A2A CALL] Connecting to support_agent at http://localhost

### Scenario 3: Multi-Step Coordination (Complex Query)

Query: Show all active customers who have closed tickets

Expected Flow:
1. Router decomposes query into sub-tasks
2. Router calls Customer Data Agent to get active customers
3. Router calls Customer Data Agent again for ticket history
4. Router filters and synthesizes final report

In [ ]:
query_3 = "Show me the names of all active customers who have closed tickets."

result_3 = await router.process_query(query_3)
print(f"\nFINAL RESPONSE:\n{result_3}")


 USER QUERY: Show me the names of all active customers who have closed tickets. 

[ROUTER STEP 1]: The user is asking for a list of customers who meet certain conditions. I should use the 'customer_data' agent to first get all the active customers and then retrieve their ticket histories. I will then filter the list to only include active customers with closed tickets.
   >>> [A2A CALL] Connecting to customer_data at http://localhost:10020...
   <<< [A2A RESPONSE]: [CUSTOMER_DATA_AGENT]: Calling list_customers with status active. ```json {"customer": [{"id": 4, "name": "Alice Williams", "email": "alice.w@techcorp.com", "phone": "+1-555-0104", "status": "active",...

[ROUTER STEP 2]: Now that I have a list of active customers, I need to get their ticket histories to filter the customers who have closed tickets. I will send a request to 'customer_data' for the ticket history of all these customers.
   >>> [A2A CALL] Connecting to customer_data at http://localhost:10020...
   <<< [A2A RE

## Summary

The demo successfully demonstrated:
- Simple queries routed to single agents
- Complex queries coordinated between multiple agents
- Multi-step coordination with data aggregation
- A2A communication via HTTP/JSON-RPC
- LLM-based query routing and decision making